[]


In [6]:
import requests
import pandas as pd
import time

# API Configuration
url = "https://clutch-co-scraper1.p.rapidapi.com/"
headers = {
    "x-rapidapi-key": "29176455d0msh76cff296476b1c5p160124jsn18fe314a32b4",
    "x-rapidapi-host": "clutch-co-scraper1.p.rapidapi.com",
    "Content-Type": "application/json"
}

# Initialize dataset storage
all_companies = []
max_pages = 10  # Adjust this based on the number of results available

# Loop through pages
for page in range(1, max_pages + 1):
    print(f"Fetching page {page}...")
    
    payload = {
        "api_type": "search",
        "search_term": "web design",
        "page": str(page)
    }
    
    response = requests.post(url, json=payload, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        companies = data.get("results", [])
        
        if not companies:
            print(f"No more data on page {page}. Stopping.")
            break  # Stop if no more data is returned
        
        all_companies.extend(companies)  # Add new data
        
    else:
        print(f"Error on page {page}: {response.status_code}")
        break  # Stop on error

    time.sleep(1)  # Prevent hitting API rate limits

# Convert to DataFrame
df = pd.DataFrame(all_companies)

# Selecting important columns (Modify based on actual response structure)
columns_to_keep = ['company_name', 'rating', 'review_count', 'location', 'services', 'website']
df = df[columns_to_keep] if all(col in df.columns for col in columns_to_keep) else df

# Save dataset
df.to_csv("clutch_web_design_companies_large.csv", index=False)

print(f"Dataset saved: {len(df)} entries in clutch_web_design_companies_large.csv")


Fetching page 1...


AttributeError: 'list' object has no attribute 'get'

In [ ]:
import requests
import csv
import time
import json

# API Configuration
url = "https://clutch-co-scraper1.p.rapidapi.com/"
headers = {
    "x-rapidapi-key": "29176455d0msh76cff296476b1c5p160124jsn18fe314a32b4",
    "x-rapidapi-host": "clutch-co-scraper1.p.rapidapi.com",
    "Content-Type": "application/json"
}

# CSV File Setup
csv_filename = "clutch_web_design_companies_large.csv"
fields = ["Company Name", "Location", "Rating", "Reviews", "Website", "Hourly Rate", "Employees", "Services"]

with open(csv_filename, "w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(fields)

    page = 1
    total_entries = 0

    while True:
        print(f"📢 Fetching page {page}...")

        # API Request
        payload = {"api_type": "search", "search_term": "web design", "page": str(page)}
        response = requests.post(url, json=payload, headers=headers)

        if response.status_code != 200:
            print(f"❌ Error: {response.status_code}. Stopping.")
            break

        data = response.json()

        # 🛠️ Debug: Print response format on first page
        if page == 1:
            print("\n📌 API Response (Page 1):")
            print(json.dumps(data, indent=2))  # Pretty print JSON

        # ✅ Check if response is a dictionary with "results" key
        if not isinstance(data, dict) or "results" not in data:
            print(f"⚠️ Unexpected response format on page {page}. Stopping.")
            break

        companies = data["results"]

        if not companies:  # If no more results
            print(f"✅ No more data on page {page}. Stopping.")
            break

        # Extract & Save Data
        for company in companies:
            row = [
                company.get("company_name", "N/A"),
                company.get("location", "N/A"),
                company.get("rating", "N/A"),
                company.get("reviews", "N/A"),
                company.get("website", "N/A"),
                company.get("hourly_rate", "N/A"),
                company.get("employees", "N/A"),
                ", ".join(company.get("services", [])) if isinstance(company.get("services"), list) else "N/A"
            ]
            writer.writerow(row)
            total_entries += 1

        print(f"✅ Page {page} processed. Total entries so far: {total_entries}")

        page += 1  # Move to next page
        time.sleep(2)  # Avoid rate-limiting (adjust if needed)

print(f"\n🎉 Dataset saved: {total_entries} entries in {csv_filename}")


📢 Fetching page 1...
